In [2]:
import numpy as np
import picos as pcs
import scipy

# output the Pauli X matrix for the i-th qubit in an n-qubit system
def X(n, i):
    matrices = [np.eye(2)] * n
    matrices[i] = np.array([[0, 1], [1, 0]])
    result = np.kron(matrices[0], matrices[1])
    for j in range(2, n):
        result = np.kron(result, matrices[j])
    return result

# output the Pauli Y matrix for the i-th qubit in an n-qubit system
def Y(n, i):
    matrices = [np.eye(2)] * n
    matrices[i] = np.array([[0, -1j], [1j, 0]])
    result = np.kron(matrices[0], matrices[1])
    for j in range(2, n):
        result = np.kron(result, matrices[j])
    return result

# output the Pauli Z matrix for the i-th qubit in an n-qubit system
def Z(n, i):
    matrices = [np.eye(2)] * n
    matrices[i] = np.array([[1, 0], [0, -1]])
    result = np.kron(matrices[0], matrices[1])
    for j in range(2, n):
        result = np.kron(result, matrices[j])
    return result

# output the EPR Hamiltonian with weight matrix M
def epr_ham(M):
    M = np.maximum(M, M.T)
    n = M.shape[0]
    H = np.zeros((2**n, 2**n), dtype=complex)
    
    for i in range(n):
        for j in range(i+1, n):
            if M[i, j] != 0:
                H += M[i, j] * (np.eye(2**n) + X(n, i) @ X(n, j) - Y(n, i) @ Y(n, j) + Z(n, i) @ Z(n, j))/4
    return H

# output all pairwise energies of the EPR Hamiltonian 
def all_energies(rho):
    n = int(np.log2(rho.shape[0]))
    energies = []
    for i in range(n):
        for j in range(i+1, n):
            # calculate the energy of the i-j pair
            M = np.zeros((n, n))
            M[i, j] = 1
            energy = np.vdot(rho, epr_ham(M)).real
            energies.append(energy)

    return energies

# output the adjacency matrix of the cycle graph with n vertices
def cycle_graph(n):
    M = np.zeros((n, n))
    for i in range(n):
        M[i, (i+1)%n] = 1
    return np.maximum(M, M.T)

# output the adjacency matrix of the path graph with n vertices
def path_graph(n):
    M = np.zeros((n, n))
    for i in range(n-1):
        M[i, i+1] = 1
    return np.maximum(M, M.T)

In [3]:
# given a hamiltonian H, we will use an SDP to compute the maximum energy state of H subject to the condition that the partial trace onto every individual qubit is equal to [[p, 0], [0, 1-p]]
# we will use the following SDP:
# maximize Tr(HX)
# subject to:
# X >= 0
# Tr(X) = 1
# Tr_i(X) = [[p, 0], [0, 1-p]] for all i
# where Tr_i is the partial trace onto the i-th qubit

def compute_state_rdm_constraint(H, p):
    n = int(np.log2(H.shape[0]))
    sdp = pcs.Problem()
    
    rho = pcs.HermitianVariable('rho', 2**n)
    sdp.add_constraint(pcs.trace(rho) == 1)
    sdp.add_constraint(rho >> 0)
    
    qubit_rdm = np.array([[p, 0], [0, 1-p]])
    # add the partial trace constraints
    for i in range(n):
        remaining = [j for j in range(n) if j != i]
        sdp.add_constraint(pcs.partial_trace(rho, subsystems=remaining) == qubit_rdm)
    
    obj = pcs.trace(H * rho).real
    sdp.set_objective('max', obj)
    sol = sdp.solve(solver='mosek',verbosity=-1)
    # print(f"Solved with {sol.claimedStatus} status")
    # print(f"Value: {obj}")
    return obj, rho


In [13]:
# For the proof of Lemma 3.2, we first need the following for k=3,5:
# There exists a k-qubit state rho_k such achieving (1) energy >= 0.6068 > (3 Sqrt[5] + 3)/16 + 1e-6 on edges of the form (i, i+1%k) and (2) energy >= 0.4046 > (1+Sqrt[5])/8 + 1e-6 on all edges (i, j).
# Furthermore, (3) the 1-qubit marginals of rho_k should be equal to Theta |0><0| + (1-Theta) |1><1| for Theta > 1/2 that satisfies Sqrt[Theta(1-Theta)] = (Sqrt[5]-1)/4, i.e., Theta = (2+Sqrt[2(Sqrt[5]-1)])/4

for k in [3, 5]:
    H = epr_ham(cycle_graph(k))
    Theta = (2+np.sqrt(2*(np.sqrt(5)-1)))/4
    obj, rho = compute_state_rdm_constraint(H, Theta)
    assert(obj/k >= 0.6068) # condition 1
    assert(np.min(all_energies(rho)) >= 0.4046) # condition 2
    for i in range(k):
        trace_out = [j for j in range(k) if j != i]
        rdm = pcs.partial_trace(rho, subsystems=trace_out).np
        assert(np.allclose(rdm, Theta*np.array([[1, 0], [0, 0]]) + (1-Theta)*np.array([[0, 0], [0, 1]]), rtol=0, atol=1e-8)) # condition 3 (all 1-RDMs are entrywise within 1e-8 of the target, so one can replace rho with an appropriate product state with probability << 1e-6 to maintain conditions (1) and (2) while ensuring condition (3) holds exactly)


In [14]:
# we also need to find a state psi on 5 qubits that satisfies (1) average energy on edges (1,2),(2,3),(3,4),(4,5) is at least 0.668 + 1e-6, (2) energy >= 0.4046 > (1+Sqrt[5])/8 + 1e-6 on all edges (i, j).
# Furthermore, (3) the 1-qubit marginals of rho_k should be equal to Theta |0><0| + (1-Theta) |1><1| for Theta > 1/2 that satisfies Sqrt[Theta(1-Theta)] < (Sqrt[5]-1)/4, i.e., Theta > 0.8931.
H = epr_ham(path_graph(5))
Theta = 0.8931
obj, rho = compute_state_rdm_constraint(H, Theta)
assert(obj/4 >= 0.669) # condition 1
assert(np.min(all_energies(rho)) >= 0.4046) # condition 2
for i in range(5):
    trace_out = [j for j in range(5) if j != i]
    rdm = pcs.partial_trace(rho, subsystems=trace_out).np
    assert(np.allclose(rdm, Theta*np.array([[1, 0], [0, 0]]) + (1-Theta)*np.array([[0, 0], [0, 1]]), rtol=0, atol=1e-8)) # condition 3 (all 1-RDMs are entrywise within 1e-8 of the target, so one can replace rho with an appropriate product state with probability << 1e-6 to maintain conditions (1) and (2) while ensuring condition (3) holds exactly)
